In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

**<font size="6" color="red">ch3. 연관분석</font>**
- pip install apyori
# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고 그 속성들 사이에 연관성이 어느 정도 잇는지 분석
- 활용분야: 이벤트미리감지(사기적발...), 신상품카테고리 구성

[조건: left-hand-side:오렌지주스] -> [결과 right-hand-side:와인]
- 연관분석과 관련된 지표
- 연과분석과 관련된 지표
1. 지지도(support) : 얼마나 자주 함께 나타나는지
    (lhs, rhs)의 항목수/전체항목수 = 0.2
    
2. 신뢰도(confidence) : 조건이 오면 결과가 얼마나 자주 나타나는지
    (lhs->rhs)의 항목수/lhs의 항목수 = 1/2 = 0.5
    
3. 향상도(lift) : 우연히 발생한 규칙은 아닌지 확인
    lhs->rhs의 지지도 / (lhs의 지지도 * rhs의 지지도)
    => 0.2 / (0.4*0.6) = 0.2/0.24 = 0.833
    향상도<1 : 기대가 낮다
    향상도>1 : 기대가 높다

# 2. 연관분석 구현

In [2]:
import csv
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    # print(list(csvdata))
    transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [3]:
from apyori import apriori
rules = apriori(transaction, # 2차원 데이터
               min_support=0.15,
               min_confidence=0.1)
rules = list(rules)
len(rules)

18

In [4]:
rules[10]

RelationRecord(items=frozenset({'콜라', '소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콜라', '소주'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

In [5]:
rule = rules[10]
support = rule[1]
order_st = rule[2]
for item in order_st:
    lhs = item[0]
    rhs = item[1]
    confidence = item[2]
    lift = item[3]
    if lift > 1:
        print("{}=>{}\t {}\t {}\t {}".format(lhs, rhs, support, 
                                             round(confidence,2), 
                                             round(lift,2)))

frozenset({'소주'})=>frozenset({'콜라'})	 0.6	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'소주'})	 0.6	 0.75	 1.25


In [6]:
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = item[0]
        rhs = item[1]
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            print("{}=>{}\t {}\t {}\t {}".format(lhs, rhs, support, 
                                                 round(confidence,2), 
                                                 round(lift,2)))

frozenset({'맥주'})=>frozenset({'콜라'})	 0.4	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'맥주'})	 0.4	 0.5	 1.25
frozenset({'소주'})=>frozenset({'콜라'})	 0.6	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'소주'})	 0.6	 0.75	 1.25
frozenset({'콜라'})=>frozenset({'소주', '맥주'})	 0.2	 0.25	 1.25
frozenset({'소주', '맥주'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25
frozenset({'맥주'})=>frozenset({'콜라', '와인'})	 0.2	 0.5	 1.25
frozenset({'콜라'})=>frozenset({'와인', '맥주'})	 0.2	 0.25	 1.25
frozenset({'와인', '맥주'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25
frozenset({'콜라', '와인'})=>frozenset({'맥주'})	 0.2	 0.5	 1.25
frozenset({'소주'})=>frozenset({'콜라', '오렌지주스'})	 0.2	 0.33	 1.67
frozenset({'콜라'})=>frozenset({'소주', '오렌지주스'})	 0.2	 0.25	 1.25
frozenset({'소주', '오렌지주스'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25
frozenset({'콜라', '오렌지주스'})=>frozenset({'소주'})	 0.2	 1.0	 1.67
frozenset({'콜라'})=>frozenset({'소주', '와인'})	 0.2	 0.25	 1.25
frozenset({'소주', '와인'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25


In [7]:
import pandas as pd
rules_df = pd.DataFrame(None, col인','오렌지', 0.15, 0.5, 1.1] 식으로 for문내에서 데이터추가
idx = 0umns=['lhs','rhs','지지도','신뢰도','향상도'])
#rules_df.loc[0] = ['와
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift,2)]
            idx += 1
rules_df.sort_values(by=['향상도','신뢰도'], ascending=False)

,lhs,rhs,지지도,신뢰도,향상도
13,"콜라, 오렌지주스",소주,0.2,1.00,1.67
10,소주,"콜라, 오렌지주스",0.2,0.33,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"소주, 맥주",콜라,0.2,1.00,1.25
8,"와인, 맥주",콜라,0.2,1.00,1.25
12,"소주, 오렌지주스",콜라,0.2,1.00,1.25
15,"소주, 와인",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25


# 3. 경주/전주 여행자료 연관분석

In [8]:
import pandas as pd
from konlpy.tag import Hannanum, Kkma, Komoran
df = pd.read_csv('data/naver_kin.csv', sep='\t')
total_text_list = df['total_text'].to_list()
# total_text_list[:2]
analyzer = Komoran()
total_noun_list = []
select_pos = ['NC','NQ'] # Hannanum 보통명사, 고유명사
select_pos = ['NNP', 'NNG']# Kkma, Komoran 보통명사, 고유명사
불용어 = {'여행'}
for total_text in total_text_list:
    #total_noun = analyzer.nouns(total_text)
    total_noun = [token for token, tag in analyzer.pos(total_text)
                     if tag in select_pos and
                         token not in 불용어 and
                         len(token)>1]
    total_noun_list.append(total_noun)
print(total_noun_list[:2])

[['전주', '한곳', '추천', '추억', '체험', '감성', '추억', '테마', '박물관', '시간', '전북', '전북', '투어', '패스', '통합', '이용', '여행지', '체험', '카페', '이용', '추가', '전주', '필수', '편안', '감사'], ['전주', '전주', '사람과', '호텔', '가격', '얼마', '정도', '맛집', '카페', '추천', '안녕하세요', '전주', '계획', '한옥마을', '근처']]


In [9]:
%%time
rules = apriori(total_noun_list, min_support=0.15, min_confidence=0.3)
rules = list(rules)
len(rules)

CPU times: total: 78.1 ms
Wall time: 89.7 ms


193

In [10]:
rules_df = pd.DataFrame(None, columns=['lhs','rhs','지지도','신뢰도','향상도'])
#rules_df.loc[0] = ['와인','오렌지', 0.15, 0.5, 1.1] 식으로 for문내에서 데이터추가
idx = 0
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift,2)]
            idx += 1
rules_df.sort_values(by=['향상도','신뢰도'], ascending=False, inplace=True)
rules_df = rules_df.reset_index(drop=True)

In [11]:
pd.options.display.max_rows

60

In [13]:
rules_df.iloc[-10:,]

,lhs,rhs,지지도,신뢰도,향상도
3226,"전주, 맛집",코스,0.197,0.78,1.53
3227,코스,"전주, 맛집",0.197,0.39,1.53
3228,맛집,코스,0.217,0.71,1.38
3229,코스,맛집,0.217,0.42,1.38
3230,전주,코스,0.296,0.59,1.16
3231,코스,전주,0.296,0.58,1.16
3232,추천,코스,0.226,0.52,1.02
3233,추천,경주,0.221,0.51,1.02
3234,경주,추천,0.221,0.44,1.02
3235,코스,추천,0.226,0.44,1.02
